In [1]:
# propitiatory invocation (i.e. the user hasn't installed oifits as a package)
import sys
sys.path.append('..') 

import numpy as np

## Opening OIFITS files

We import the oifits module and open sample OIFITS files.

In [2]:
import pyoifits as oifits

# OIFITS version 2 from GRAVITY @ VLTI
gravity = oifits.open('test1.fits')
# OIFITS version 1 from PIONIER @ VLTI
pionier = oifits.open('test2.fits')

# The GRAVITY pipeline produces a non-standard OI_FLUX table, we fix that.
for h in gravity[8::4]:
    h.rename_columns(FLUX='FLUXDATA')

Verify the compliance to the standard and fix mendable issues.

In [3]:
gravity.verify('fix+warn')
pionier.verify('fix+warn')

 [astropy.io.fits.verify]


The OIFITS instances print nicely, indicating extensions with the data dimension.

In [4]:
gravity

<OIFITS2 at 0x7fbee2d0a270: <PrimaryHDU2 (void)> <ArrayHDU2 (6C×4R)> <TargetHDU2 (17C×2R)> <WavelengthHDU2 (2C×210R=210W)> <WavelengthHDU2 (2C×5R=5W)> <VisHDU2 (20C×6R×5W)> <Vis2HDU2 (12C×6R×5W)> <T3HDU2 (16C×4R×5W)> <FluxHDU1 (10C×4R×5W)> <VisHDU2 (29C×6R×210W)> <Vis2HDU2 (12C×6R×210W)> <T3HDU2 (16C×4R×210W)> <FluxHDU1 (15C×4R×210W)>>

In [5]:
pionier

<OIFITS1 at 0x7fbe93011ea0: <PrimaryHDU1 (void)> <TargetHDU1 (17C×1R)> <WavelengthHDU1 (2C×3R=3W)> <ArrayHDU1 (5C×4R)> <Vis2HDU1 (10C×5R×3W)> <T3HDU1 (14C×4R×3W)>>

OIFITS files can be merged easily.  Reindexing of extensions and target/station indices are taken care of.

In [6]:
# we merge arrays and stations of the same name, not caring about their exact
# coordinates. Reason: PIONIER sets them to zero but we know both PIONIER and 
# GRAVITY name the stations correctly.
oifits.set_merge_settings(array_distance=1e+9, station_distance=1e+9)
merged = gravity + pionier
merged

<OIFITS2 at 0x7fbe92ea02c0: <PrimaryHDU2 (void)> <TargetHDU2 (17C×3R)> <ArrayHDU2 (6C×7R)> <WavelengthHDU2 (2C×5R=5W)> <WavelengthHDU2 (2C×210R=210W)> <WavelengthHDU2 (2C×3R=3W)> <VisHDU2 (20C×6R×5W)> <VisHDU2 (29C×6R×210W)> <Vis2HDU2 (12C×6R×5W)> <Vis2HDU2 (12C×6R×210W)> <Vis2HDU2 (10C×5R×3W)> <T3HDU2 (16C×4R×5W)> <T3HDU2 (16C×4R×210W)> <T3HDU2 (14C×4R×3W)> <FluxHDU1 (10C×4R×5W)> <FluxHDU1 (15C×4R×210W)>>

In [7]:
merged[0].header[0:19]

SIMPLE  =                    T / file does conform to FITS standard             
BITPIX  =                    8 / array data type                                
NAXIS   =                    0 / number of data axes                            
EXTEND  =                    T / FITS dataset may contain extensions            
COMMENT   FITS (Flexible Image Transport System) format is defined in 'Astronomy
COMMENT   and Astrophysics', volume 376, page 359; bibcode: 2001A&A...376..359H 
DATE    = '2020-07-02T19:52:50' / file creation date (YYYY-MM-DDThh:mm:ss UT)   
DATE-OBS= '2012-12-21T04:12:03.022' / Observing date                            
CONTENT = 'OIFITS2 '                                                            
REFERENC= 'MULTI   '                                                            
OBSERVER= 'UNKNOWN '           / Name of observer.                              
PROG_ID = 'MULTI   '                                                            
PROCSOFT= 'MULTI   '        

One of the problems of the OIFITS standard is how complicated the data structure is, with a lot of cross-references.  It is possible to obtain the data in a flat table, listing one single scalar observable per line together to all relevant parameters such as target, date, wavelength, band, baseline(s).

In [8]:
tab = merged.to_table()
print(tab['TARGET','EFF_WAVE','EFF_BAND','U1COORD','V1COORD','U2COORD','V2COORD','observable','value','error'])

 TARGET    EFF_WAVE    EFF_BAND  U1COORD ... observable  value    error  
-------- ----------- ----------- ------- ... ---------- ------- ---------
CO_Ori_A 2.02369e-06 8.50000e-08   5.488 ...     VISAMP 0.84688 0.0041681
CO_Ori_A 2.09294e-06 8.50000e-08   5.488 ...     VISAMP 0.94783 0.0019325
CO_Ori_A 2.19100e-06 8.50000e-08   5.488 ...     VISAMP 0.97285   0.00123
CO_Ori_A 2.29188e-06 8.50000e-08   5.488 ...     VISAMP  0.9789 0.0017413
CO_Ori_A 2.36233e-06 8.50000e-08   5.488 ...     VISAMP 0.96522 0.0013306
CO_Ori_A 2.02369e-06 8.50000e-08  -2.729 ...     VISAMP 0.72994 0.0063785
CO_Ori_A 2.09294e-06 8.50000e-08  -2.729 ...     VISAMP 0.80576 0.0026422
CO_Ori_A 2.19100e-06 8.50000e-08  -2.729 ...     VISAMP 0.86026 0.0021998
CO_Ori_A 2.29188e-06 8.50000e-08  -2.729 ...     VISAMP 0.89297 0.0013147
CO_Ori_A 2.36233e-06 8.50000e-08  -2.729 ...     VISAMP 0.92514 0.0019255
     ...         ...         ...     ... ...        ...     ...       ...
CO_Ori_B 2.42799e-06 4.40191e-09      

## Creating OIFITS files

We start by creating an array HDU.  The local station coordinates can be provided (East-North-Up) instead of the XYZ geocentric format of the OIFITS. They will be converted (WGS84).

In [9]:
# WGS coordinates of nominal VLTI centre  and nominal station positions.
# (InterfaceControl Document between VLTI and its Instruments (Part I)
# Document ESO-045686, v. 7.3)
arrname = 'VLTI'
lat = -24.62743941
lon = -70.40498689
alt = 2669
sta_name = ['A0', 'B1', 'J1', 'J6']
tel_name = ['AT1', 'AT2', 'AT3', 'AT4']
diameter = [1.8, 1.8, 1.8, 1.8]
sta_enu = [[-14.642, -55.812, 4.54],
           [ -1.863, -68.334, 4.54],
           [106.648, -39.444, 4.54],
           [ 59.810,  96.706, 4.54]]

# Build the OI_ARRAY table 
array  = oifits.new_array_hdu(arrname, lat=lat, lon=lon, alt=alt,
            tel_name=tel_name, sta_name=sta_name, sta_enu=sta_enu,
            diameter=1.8)
print(array.STAXYZ)


[[-20.20989991  13.11233089 -52.62704469]
 [ -9.92097336  22.31393991 -64.01000155]
 [ 96.34328172  47.36351202 -37.74793289]
 [ 71.24532488 -21.79466993  86.01740627]]


OI_TARGET HDUs can be created in the same way or directly from the SIMBAD identifiers. Non ascii-names will be substituted.

In [10]:
simbad_id = ['beta Car', 'η Car']
category = ['SCI', 'CAL']
target = oifits.new_target_hdu_from_simbad(simbad_id, category=category)
target.data


FITS_rec([(1, 138.29990608, -69.7172076 , 2000., 4.70269285e-09, 5.96320837e-09,  -5100., 'BARYCENT', 'OPTICAL', -7.58587967e-07, 5.28204506e-07, 6.78739156e-10, 5.33295046e-10, 1.397233e-07, 5.33295e-10, 'beta Car', 'A1III-', 'SCI'),
          (2, 161.26477294, -59.68443085, 2000., 4.84813681e-08, 5.33295049e-08, -25000., 'BARYCENT', 'OPTICAL', -5.33295049e-08, 1.98773609e-08, 3.39369571e-09, 3.87850951e-09,          nan,         nan, '* eta Car', 'OBepec', 'CAL')],
         dtype=(numpy.record, [('TARGET_ID', '<i2'), ('RAEP0', '<f8'), ('DECEP0', '<f8'), ('EQUINOX', '<f4'), ('RA_ERR', '<f8'), ('DEC_ERR', '<f8'), ('SYSVEL', '<f8'), ('VELTYP', 'S8'), ('VELDEF', 'S8'), ('PMRA', '<f8'), ('PMDEC', '<f8'), ('PMRA_ERR', '<f8'), ('PMDEC_ERR', '<f8'), ('PARALLAX', '<f4'), ('PARA_ERR', '<f4'), ('TARGET', 'S32'), ('SPECTYP', 'S32'), ('CATEGORY', 'S3')]))

Create OI_WAVELENGTH and OI_VIS2 tables

In [11]:
insname = 'TESTING-3CHANNELS'
wave = [2.0e-6, 2.2e-6, 2.4e-6]
band = [0.2e-6, 0.2e-6, 0.2e-6]
wavelength = oifits.new_wavelength_hdu(insname, eff_wave=wave, eff_band=band)

In [12]:
sta_index = [[1, 2], [1, 3], [1,4], [2,3], [2,4], [3, 4]] * 2
target_id = [1] * 6 + [2] * 6
vis2data = [[1, 1, 1]] * 12
vis2err = [[0.05, 0.05, 0.05]] * 12
mjd = np.linspace(58880, 58880.2, 12)

vis2 = oifits.new_vis2_hdu(
        insname, arrname=arrname, mjd=mjd,
        target_id=target_id, sta_index=sta_index,
        vis2data=vis2data, vis2err=vis2err)

In [18]:
obs = oifits.OIFITS2([oifits.PrimaryHDU2(), target, array, wavelength, vis2])
obs

<OIFITS2 at 0x7fbe92a6b040: <PrimaryHDU2 (void)> <TargetHDU2 (18C×2R)> <ArrayHDU2 (5C×4R)> <WavelengthHDU2 (2C×3R=3W)> <Vis2HDU2 (10C×12R×3W)>>

In [19]:
obs.verify('warn')

 [astropy.io.fits.verify]
